In [1]:
import pandas as pd
data=pd.read_csv('creditcard2013.csv')

In [2]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
data['Amount']=sc.fit_transform(pd.DataFrame(data['Amount']))
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [3]:
data=data.drop(['Time'],axis=1)

In [4]:
data=data.drop_duplicates()

In [5]:
data['Class'].value_counts()

0    275190
1       473
Name: Class, dtype: int64

In [6]:
x=data.drop('Class',axis=1)
y=data['Class']

In [7]:
from sklearn.model_selection import train_test_split
X_train ,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [8]:
x.shape

(275663, 29)

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU

# Assuming 'data' is your dataset
# Split dataset into features (X) and labels (y)
x = data.drop('Class', axis=1)
y = data['Class']

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Get minority class samples
minority_class = 1
minority_X = X_train[Y_train == minority_class]

# Define the generator model
def build_generator(latent_dim, output_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(output_dim, activation='tanh'))  # Output layer with tanh
    return model

# Define the discriminator model
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification (real/fake)
    return model

# Build the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator when training the GAN model
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Function to train the GAN
def train_gan(generator, discriminator, gan_model, real_samples, latent_dim, n_epochs=1000, batch_size=64):
    for epoch in range(n_epochs):
        # Select a random batch of real samples
        idx = np.random.randint(0, real_samples.shape[0], batch_size)
        real_batch = real_samples[idx]
        real_labels = np.ones((batch_size, 1))  # Real labels for discriminator

        # Generate a batch of fake samples
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_batch = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))  # Fake labels for discriminator

        # Train the discriminator
        discriminator.train_on_batch(real_batch, real_labels)
        discriminator.train_on_batch(fake_batch, fake_labels)

        # Train the generator via the GAN
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))  # Valid labels for generator
        gan_model.train_on_batch(noise, valid_y)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{n_epochs} completed.")

# Function to generate synthetic samples
def generate_samples(generator, n_samples, latent_dim):
    noise = np.random.normal(0, 1, (n_samples, latent_dim))
    synthetic_samples = generator.predict(noise)
    return synthetic_samples

# Build and compile the models
latent_dim = 20  # Latent dimension (size of input noise)
generator = build_generator(latent_dim, X_train.shape[1])
discriminator = build_discriminator(X_train.shape[1])
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# Build the GAN model and compile it
gan_model = build_gan(generator, discriminator)
gan_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss='binary_crossentropy')

# Train the GAN model
train_gan(generator, discriminator, gan_model, minority_X, latent_dim, n_epochs=1000, batch_size=64)

# Generate synthetic samples to balance the classes
majority_class_samples = X_train[Y_train == 0]

# Calculate the number of synthetic samples needed to match the majority class
n_synthetic_samples = majority_class_samples.shape[0] - minority_X.shape[0]

# Generate enough synthetic samples to match the number of majority class samples
if n_synthetic_samples > 0:
    synthetic_samples = generate_samples(generator, n_synthetic_samples, latent_dim)
else:
    # If the minority class is larger than the majority, generate enough to balance them
    synthetic_samples = generate_samples(generator, majority_class_samples.shape[0], latent_dim)

# Combine synthetic samples with original data
X_resampled = np.vstack((X_train, synthetic_samples))
y_resampled = np.hstack((Y_train, np.full(synthetic_samples.shape[0], minority_class)))

# Train classifier on resampled data
clf = RandomForestClassifier()
clf.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model performance
print(classification_report(Y_test, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


/Users/mayankpathak/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/mayankpathak/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/mayankpathak/anaconda3/lib/python3.11/site-packages/keras/src/backend/tensorflow/trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/1000 completed.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
2/2 

In [10]:
num_zeros = np.sum(y_resampled == 0)
num_ones = np.sum(y_resampled == 1)

print(f"Number of 0s: {num_zeros}")
print(f"Number of 1s: {num_ones}")

Number of 0s: 220148
Number of 1s: 220148


In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

 

# Initialize the LinearSVC model
linear_svc_model = LinearSVC(C=1.0, max_iter=1000, random_state=42)

# Train the model on the resampled training data
linear_svc_model.fit(X_resampled,y_resampled)

# Make predictions on the test data
y_pred = linear_svc_model.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(Y_test, y_pred, target_names=['Not Fraud', 'Fraud']))

Classification Report:
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.85      0.67      0.75        91

    accuracy                           1.00     55133
   macro avg       0.92      0.84      0.87     55133
weighted avg       1.00      1.00      1.00     55133



In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
log=LogisticRegression()
log.fit(X_resampled,y_resampled)
y_pred=log.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.76      0.66      0.71        91

    accuracy                           1.00     55133
   macro avg       0.88      0.83      0.85     55133
weighted avg       1.00      1.00      1.00     55133



In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_resampled,y_resampled)

# Make predictions
y_pred = knn.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.91      0.77      0.83        91

    accuracy                           1.00     55133
   macro avg       0.95      0.88      0.92     55133
weighted avg       1.00      1.00      1.00     55133



In [20]:
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
dt = DecisionTreeClassifier(criterion='entropy', max_depth=50, random_state=42)
dt.fit(X_resampled,y_resampled)
y_pred = dt.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.73      0.73      0.73        91

    accuracy                           1.00     55133
   macro avg       0.86      0.86      0.86     55133
weighted avg       1.00      1.00      1.00     55133



In [25]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 99.91%


In [15]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=100, max_depth=50, learning_rate=0.1, random_state=42)
xgb_model.fit(X_resampled,y_resampled)

# Make predictions
y_pred = xgb_model.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.94      0.73      0.82        91

    accuracy                           1.00     55133
   macro avg       0.97      0.86      0.91     55133
weighted avg       1.00      1.00      1.00     55133



In [24]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 99.91%


In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

 

# Assuming you have already performed GAN oversampling and have X_resampled and y_resampled
# Build the Multi-Layer Perceptron (MLP) model
def build_mlp(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))  # First hidden layer
    model.add(Dropout(0.2))  # Dropout for regularization
    model.add(Dense(64, activation='relu'))  # Second hidden layer
    model.add(Dropout(0.2))  # Dropout for regularization
    model.add(Dense(32, activation='relu'))  # Third hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer (sigmoid for binary classification)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the MLP model using resampled data
mlp_model = build_mlp(X_resampled.shape[1])

# Fit the model on resampled data (X_resampled and y_resampled)
mlp_model.fit(X_resampled, y_resampled, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Make predictions on the test set
y_pred = (mlp_model.predict(X_test) > 0.5).astype(int)  # Converting probabilities to class labels

# Print classification report
print(classification_report(Y_test, y_pred, target_names=['Not Fraud', 'Fraud']))


/Users/mayankpathak/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 5s 782us/step - accuracy: 0.9944 - loss: 0.0182 - val_accuracy: 1.0000 - val_loss: 8.3967e-09
Epoch 2/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 771us/step - accuracy: 0.9996 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 7.0160e-10
Epoch 3/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 772us/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 1.0731e-10
Epoch 4/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 764us/step - accuracy: 0.9996 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 8.2642e-09
Epoch 5/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 779us/step - accuracy: 0.9997 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 6.2736e-17
Epoch 6/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 762us/step - accuracy: 0.9996 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 7.9251e-14
Epoch 7/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 763us/step - accuracy: 0.9996 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 3.3985e-14
Epoch 8/50
5504/5504 ━━━━━━━━━━━━━━━━━━━━ 4s 762

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42)
rf_model.fit(X_resampled,y_resampled)
y_pred = rf_model.predict(X_test)
print (classification_report(Y_test,y_pred, target_names=['Not Fraud', 'Fraud']))

              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00     55042
       Fraud       0.96      0.74      0.83        91

    accuracy                           1.00     55133
   macro avg       0.98      0.87      0.92     55133
weighted avg       1.00      1.00      1.00     55133



In [23]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 99.91%
